In [1]:
# Downloading necessary files
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import mplfinance as mpf 
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from datetime import datetime
import plotly.graph_objects as go
import re
import json
import requests 
import codecs
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.offline as pyo


# display plotly in browser when run in Spyder
# pio.renderers.default = 'browser'

In [2]:
# stock =  input('Type in the stock symbol: ').upper()
stock = 'TSLA'

In [3]:
def create_df():

    # =============================================================================
    # Creating a new dataset
    # =============================================================================
    
    global df
    
    start = "2009-01-01"
    end = dt.datetime.now()
    short_sma = 50
    long_sma = 200
    
    # creating a dataset for selected stock 
    df = yf.download(stock, start, end,interval = '1d')
    df = pd.DataFrame(df.dropna(), columns = ['Open', 'High','Low','Close', 'Adj Close', 'Volume'])


    # preparing data from time series analysis
    # eliminating any NAs 
    df.index = pd.to_datetime(df.index)
    df = df.asfreq('B')

    print('============================================================')
    print('Nan in each columns before Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('The df has {} rows and {} columns.'.format(*df.shape))
    
    df = df.bfill()
    print('============================================================')
    print('Nan in each columns after Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('The df has {} rows and {} columns.'.format(*df.shape))
#    df = df.dropna()
    
    # Create short SMA
    df['short_SMA'] = df.iloc[:,1].rolling(window = short_sma).mean()
    
    # Create Long SMA
    df['long_SMA'] = df.iloc[:,1].rolling(window = long_sma).mean()
    
    # Create daily_return
    df['daily_return'] = df['Close'].pct_change(periods=1).mul(100)
    
    # Create monthly_return
    df['monthly_return'] = df['Close'].pct_change(periods=30).mul(100)
    
    # Create annual_return
    df['annual_return'] = df['Close'].pct_change(periods=365).mul(100)
    df['Name'] = stock

    print('============================================================')
    print(stock, '- Full Dataset')
    print('------------------------------------------------------------')
    print(df.head())
    print('------------------------------------------------------------')
    print(stock, 'Full Dataset - Column Names')
    print(df.columns)

    print('============================================================')
    print('Nan in each columns after Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('============================================================')
    
#     # write to csv
#     df.to_csv(r"df.csv", index =  True)
    

In [4]:
create_df()

[*********************100%***********************]  1 of 1 completed
Nan in each columns before Backfill for TSLA
------------------------------------------------------------
Open         103
High         103
Low          103
Close        103
Adj Close    103
Volume       103
dtype: int64
The df has 2977 rows and 6 columns.
Nan in each columns after Backfill for TSLA
------------------------------------------------------------
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
The df has 2977 rows and 6 columns.
TSLA - Full Dataset
------------------------------------------------------------
             Open   High    Low  Close  Adj Close      Volume  short_SMA  \
Date                                                                       
2010-06-29  3.800  5.000  3.508  4.778      4.778  93831500.0        NaN   
2010-06-30  5.158  6.084  4.660  4.766      4.766  85935500.0        NaN   
2010-07-01  5.000  5.184  4.054  4.392      4

In [5]:
def create_y():
    
    global y
    
    y = pd.DataFrame(df['Close'], columns = ['Close'])
    y.sort_index(inplace = True)
    # dropping the first na (because there is no difference)
    y = y.dropna()
    y['Name'] = stock

    # Create SMA 16-Period for MACD
    y['SMA_12'] = df.iloc[:,1].rolling(window = 12).mean()
    
    # Create SMA 16-Period for MACD
    y['SMA_26'] = df.iloc[:,1].rolling(window = 26).mean()
    
    y['MACD'] = y['SMA_12'] -  y['SMA_26']
    
    # examining the pct_change
    y['Close Percentage Change'] = y['Close'].pct_change(1)
    
    # Creating a new variable, examining the difference for each observation
    y['diff'] = y['Close'].diff()

    # logging the target varialbe due to great variance
    y['log_Close'] = np.log(y['Close'])
    
    # Creating a new variable, examining the difference for each observation
    y['log_Close_diff'] = y['log_Close'].diff()
    
    y['Logged Close Percentage Change'] = y['log_Close'].pct_change(1)
    
    # dropping the first na (because there is no difference)
    y = y.dropna()
    

#    # write to csv
#    y.to_csv(r"y.csv", index =  True)

    print('============================================================')
    print(stock, '- Target Variable')
    print('------------------------------------------------------------')
    print(y.head())
    print('------------------------------------------------------------')
    print('Column Names')
    print(y.columns)
    print('============================================================')

#     # write to csv
#     y.to_csv(r"y.csv", index =  True)
    


In [6]:
create_y()

TSLA - Target Variable
------------------------------------------------------------
            Close  Name    SMA_12    SMA_26      MACD  \
Date                                                    
2010-08-03  4.390  TSLA  4.260500  4.244462  0.016038   
2010-08-04  4.252  TSLA  4.259333  4.222769  0.036564   
2010-08-05  4.090  TSLA  4.254333  4.154538  0.099795   
2010-08-06  3.918  TSLA  4.242000  4.110231  0.131769   
2010-08-09  3.920  TSLA  4.220833  4.086231  0.134603   

            Close Percentage Change   diff  log_Close  log_Close_diff  \
Date                                                                    
2010-08-03                 0.049235  0.206   1.479329        0.048061   
2010-08-04                -0.031435 -0.138   1.447389       -0.031940   
2010-08-05                -0.038100 -0.162   1.408545       -0.038844   
2010-08-06                -0.042054 -0.172   1.365581       -0.042964   
2010-08-09                 0.000510  0.002   1.366092        0.000510   

    

In [10]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 


trace =  go.Candlestick(
                x = df.index,
                open = df["Open"],
                high = df["High"],
                low = df["Low"],
                close = df["Close"],
                name = stock)
short_sma = go.Scatter(x = df.index,
                             y = df['short_SMA'],
                             name = 'Short SMA 50-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Short (50-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="blue"))

long_sma = go.Scatter(x = df.index,
                             y = df['long_SMA'],
                             name = 'Long SMA 200-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Long (200-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="black"))

data = [trace, short_sma, long_sma]
layout = dict(
    title = 'Price of {} (zoom in)'.format(stock),
    yaxis_title = 'US Dollars',
    yaxis_tickprefix = '$', yaxis_tickformat = ',.2f',
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
# fig.update_layout(showlegend=False)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.5,
    xanchor="right",
    x=1
))

fig.update_xaxes(
    rangeslider_visible=True,
    tickformatstops = [
        dict(dtickrange=[604800000, "M6"], value="%e. %b w"),
        dict(dtickrange=["M6", "M12"], value="%b '%y M"),
        dict(dtickrange=["M12", None], value="%Y Y")
    ],
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
    ]
)


def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - 0.25, in_view.High.max() + 0.25]

fig.layout.on_change(zoom, 'xaxis.range')
fig

FigureWidget({
    'data': [{'close': array([   4.77799988,    4.76599979,    4.3920002 , ..., 1156.86999512,
…

In [8]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 

trace = go.Bar(x = df.index,
                    y = df['Volume'],
                    name = 'Volume',
                    customdata = df['Name'],
                    hovertemplate="<b>%{customdata}</b><br><br>" +
                                    "Date: %{x|%d %b %Y} <br>" +
                                    "Volume: %{y:,.}<br>" +
                                    "<extra></extra>",
                    marker = dict(color="black"))

data = [trace]
layout = dict(
    title = 'Volume of {} (zoom in)'.format(stock),
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
fig.update_layout(showlegend=False)

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=False)


fig.update_xaxes(
    rangeslider_visible=True,
    tickformatstops = [
        dict(dtickrange=["M1", "M12"], value="%b '%y M"),
        dict(dtickrange=["M12", None], value="%Y Y")
    ],
        rangebreaks=[
        dict(bounds=["sat", "mon"])], #hide weekends
)



def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Volume.min() - 0.25, in_view.Volume.max() + 500000]

fig.layout.on_change(zoom, 'xaxis.range')
fig

FigureWidget({
    'data': [{'customdata': array(['TSLA', 'TSLA', 'TSLA', ..., 'TSLA', 'TSLA', 'TSLA'], dtype=…